In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
root = '/kaggle/input/competitive-data-science-predict-future-sales/'
# Any results you write to the current directory are saved as output.

/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv


In [2]:
train_df = pd.read_csv(os.path.join(root, 'sales_train.csv'))
train_df
test_df = pd.read_csv(os.path.join(root, 'test.csv'))
test_df

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268
...,...,...,...
214195,214195,45,18454
214196,214196,45,16188
214197,214197,45,15757
214198,214198,45,19648


Grouping shops and their items based on monthly sales 

In [3]:
for index, ele in enumerate(train_df['item_cnt_day']):
    if (ele.is_integer() == False) | (ele < 0) :
        train_df.at[index, 'item_cnt_day']  = np.NaN
train_df.dropna(inplace = True)
train_df = train_df.groupby(['date_block_num', 'shop_id', 'item_id']).agg({'item_cnt_day':np.sum}).reset_index()
train_df

,date_block_num,shop_id,item_id,item_cnt_day
0,0,0,32,6.0
1,0,0,33,3.0
2,0,0,35,1.0
3,0,0,43,1.0
4,0,0,51,2.0
...,...,...,...,...
1608221,33,59,22087,6.0
1608222,33,59,22088,2.0
1608223,33,59,22091,1.0
1608224,33,59,22100,1.0


Data visualization

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
#plt.scatter(x = train_df['shop_id'], y = train_df['item_cnt_day'])

In [5]:
#plt.scatter(x = train_df['item_id'], y = train_df['item_cnt_day'])

Applying k-NN regresion to the data

In [6]:
train_df.at[train_df['item_cnt_day'] > 500, 'item_cnt_day'] = np.NaN
train_df.dropna(inplace = True)
#plt.scatter(x = train_df['item_id'], y = train_df['item_cnt_day'])
train_df.describe()

,date_block_num,shop_id,item_id,item_cnt_day
count,1.608130e+06,1.608130e+06,1.608130e+06,1.608130e+06
mean,1.466487e+01,3.280795e+01,1.068107e+04,2.228575e+00
std,9.542431e+00,1.653657e+01,6.238654e+03,6.098824e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,6.000000e+00,2.100000e+01,5.045000e+03,1.000000e+00
50%,1.400000e+01,3.100000e+01,1.049700e+04,1.000000e+00
75%,2.300000e+01,4.700000e+01,1.606000e+04,2.000000e+00
max,3.300000e+01,5.900000e+01,2.216900e+04,5.000000e+02


In [7]:
from sklearn.neighbors import KNeighborsClassifier
#knn = KNeighborsClassifier(n_neighbors = 10, weights= 'distance')


from  sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators = 10, max_depth = 20)
train_df_copy = train_df
train_df_copy = train_df_copy.drop(labels = ['date_block_num'], axis = 1)
train_y = train_df_copy.pop('item_cnt_day')
train_df_copy.describe()

,shop_id,item_id
count,1.608130e+06,1.608130e+06
mean,3.280795e+01,1.068107e+04
std,1.653657e+01,6.238654e+03
min,0.000000e+00,0.000000e+00
25%,2.100000e+01,5.045000e+03
50%,3.100000e+01,1.049700e+04
75%,4.700000e+01,1.606000e+04
max,5.900000e+01,2.216900e+04


In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
clf_train = scaler.fit_transform(train_df_copy)
rfc.fit(clf_train,train_y)

from sklearn.model_selection import cross_val_score
scores = cross_val_score(rfc, clf_train, train_y, cv=2)
scores

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=2.
  % (min_groups, self.n_splits)), Warning)


array([0.64964678, 0.62663116])

In [9]:
test_df_copy = test_df.copy()
test_id = test_df_copy.pop('ID')
clf_test = scaler.transform(test_df_copy)
test_y = rfc.predict(clf_test)
test_y = test_y.T
res = pd.DataFrame(test_y, columns = ['item_cnt_month'])
res.to_csv('titanic.csv',index=True)